# Generate Recordings Descriptions
This service will use ChatGPT to read the recordings transcriptions and will generate meaningful description for every recording

This notebook will also query ChatGPT to provide classification of the recordings according [to the schema](../../references/ChatGPT_Prompts.md)

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import json

# for un-makrdowning the text from openai
from markdown_it import MarkdownIt
from mdit_plain.renderer import RendererPlain

from lib_henryk.config import *
from lib_henryk.logger import *
from lib_henryk import utils
from lib_henryk.recordings import transcriptions
from lib_henryk.recordings import classification
from lib_henryk.recordings import recordings

In [3]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# Your OpenAI API key
api_key = os.getenv('OPENAI_API_KEY')

## Load Resources
- prompts - there are a set of pre-configured prompts that behave well and give good results
- transcriptions - all recordings are processed with transcription service and results are saved to a parquet file (db)

In [4]:
# recordings transcriptions
df_transcriptions = pd.read_parquet(FILE_TRANSCRIPTIONS_PARQUET)
df_transcriptions = recordings.sort_df_by_date_inferred_from_name(df_transcriptions)
df_transcriptions_classifications = pd.read_parquet(FILE_TRANSCRIPTIONS_CLASSIFICATION_PARQUET)

## Perform Classification
- read existing transcriptions database (parquet file)
- read existing classification database (parquet file)
- run classifier with openai gpt (currently model 4o)
- use prompt crafted in a separate file [classification prompt](../../resources/recording_classification.md)
- we are using `few-shots` engineered prompt with rich examples

In [5]:
df_transcriptions_selected = df_transcriptions.iloc[400:]

In [6]:
transcription_classifier = classification.Transcription_Classifier(api_key=api_key)
transcription_classifier.initialise_prompt(prompt_file_path=FILE_PROMPT_RECORDING_CLASSIFICATION)

In [7]:
transcription_classifier.perform_classification(df_transcriptions_selected, df_transcriptions_classifications, verbose=False)
transcription_classifier.save_classification_parquet(path=FILE_TRANSCRIPTIONS_CLASSIFICATION_PARQUET)
df_transcriptions_classifications = transcription_classifier.df_transcriptions_classifications

2024-06-21 12:25:16 - INFO  - perform_classification - intermediate classification results will be written to /tmp/temp_sharp_rhodes.parquet
2024-06-21 12:25:16 - INFO  - perform_classification - found 306 (103) existing classifications, those transcriptions will be ignored
classifying 200 transcriptions |████████████████████████████████████████| 100.0% done.                                                                    ...
2024-06-21 13:32:35 - INFO  - perform_classification - more (403) classifications than requested, classifier was executed before on different dataset 
2024-06-21 13:32:35 - INFO  - perform_classification - *** all 200 requested transcriptions classifications were completed ***
2024-06-21 13:32:35 - INFO  - save_classification_parquet - saving 403 classification results to to ../../data/processed/henryk_transcriptions_classification.parquet
2024-06-21 13:32:35 - INFO  - save_classification_parquet - found older file with the same name, checking the older file si

In [8]:
exit_cell()

StopExecution: stopped

In [ ]:
text_md = transcription_classifier.messages.to_dict()['data'][0]['content'][0]['text']['value']
text_json = transcription_classifier.unmarkdown_parser.render(text_md)

In [ ]:
response = transcription_classifier.df_transcription_classification.iloc[-1]['classification_json']
response_json = json.loads(response)
display(response_json)

In [ ]:
transcription_classifier.save_classification_parquet(FILE_TRANSCRIPTIONS_CLASSIFICATION_PARQUET)

In [ ]:
df_backup = pd.read_parquet(FILE_TRANSCRIPTIONS_CLASSIFICATION_PARQUET)

In [ ]:
transcription_classifier.df_transcriptions_classifications = df_backup.copy()

In [ ]:
df_transcriptions_classifications = df_backup.copy()

In [ ]:
len(df_backup)